In [1]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.functions import mean,col,split, col, regexp_extract, when, lit
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.regression import LinearRegression

In [2]:
import random
import numpy as np
#set seed
random.seed(1234)
np.random.seed(1234)

In [3]:
spark = SparkSession.builder.appName("PySparkML").getOrCreate()

In [4]:
spark

## №1 Линейная регрессия

Загрузите данные для применения линейной регрессии <a href="https://github.com/AlexKbit/stepik-ds-course/raw/master/Week5/SparkML/spark-tasks/linear_regression.parquet">linear_regression.parquet</a>

In [5]:
lr_df = spark.read.parquet("./linear_regression.parquet")

In [6]:
lr_df.show(5)

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 5 rows



Создайте учителя линейной регресии со следующими параметрами:
maxIter=20
regParam=0.5
elasticNetParam=0.75<br>
<a href="https://spark.apache.org/docs/latest/ml-classification-regression.html#linear-regression">LinearRegression</a>

In [12]:
lr = LinearRegression(featuresCol = 'features', labelCol = 'label', maxIter = 20, regParam = 0.5, elasticNetParam = 0.75)

Выполните обучения на загруженных данных и сохраните результат в переменную.

In [13]:
lrModel = lr.fit(dataset = lr_df)

Найдите следующие параметры полученной модели rootMeanSquaredError (RMSE), r2 и округлити их до 3его знака.

In [39]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(labelCol = 'label', predictionCol = 'prediction', metricName = "r2")
evaluator.evaluate(lrModel.transform(lr_df))

0.01740889330766371

## №2 Кластеризация (K-Means)

Загрузите данные для применения из <a href="https://github.com/AlexKbit/stepik-ds-course/raw/master/Week5/SparkML/spark-tasks/wine.parquet">wine.parquet</a>

In [40]:
wine_df = spark.read.parquet("./wine.parquet")

In [41]:
wine_df.show(5)

+-------+----------+----+------------+---------+-------------+----------+--------------------+---------------+---------------+----+-----+-------+----------------+
|Alcohol|Malic_Acid| Ash|Ash_Alcanity|Magnesium|Total_Phenols|Flavanoids|Nonflavanoid_Phenols|Proanthocyanins|Color_Intensity| Hue|OD280|Proline|Customer_Segment|
+-------+----------+----+------------+---------+-------------+----------+--------------------+---------------+---------------+----+-----+-------+----------------+
|  14.23|      1.71|2.43|        15.6|      127|          2.8|      3.06|                0.28|           2.29|           5.64|1.04| 3.92|   1065|               1|
|   13.2|      1.78|2.14|        11.2|      100|         2.65|      2.76|                0.26|           1.28|           4.38|1.05|  3.4|   1050|               1|
|  13.16|      2.36|2.67|        18.6|      101|          2.8|      3.24|                 0.3|           2.81|           5.68|1.03| 3.17|   1185|               1|
|  14.37|      1.95| 2

Примените <a href="https://spark.apache.org/docs/latest/ml-features.html#vectorassembler">VectorAssembler</a> для создания вектора фич (задействуйте все свойства, кроме Customer_Segment).

In [62]:
feature = VectorAssembler(
    inputCols=wine_df.columns[:-1],
    outputCol="features")
wine_df = feature.transform(wine_df)

Cоздайте учителя KMeans со следующими параметрами K=3 Seed=1<br>
Обучите модель и примените ее к тому же вектору.
Документация по <a href="https://spark.apache.org/docs/latest/ml-clustering.html#k-means">KMeans</a>

In [65]:
kmeans = KMeans().setK(3).setSeed(1)
kmmodel = kmeans.fit(wine_df)


+-------+----------+----+------------+---------+-------------+----------+--------------------+---------------+---------------+----+-----+-------+----------------+--------------------+----------+
|Alcohol|Malic_Acid| Ash|Ash_Alcanity|Magnesium|Total_Phenols|Flavanoids|Nonflavanoid_Phenols|Proanthocyanins|Color_Intensity| Hue|OD280|Proline|Customer_Segment|            features|prediction|
+-------+----------+----+------------+---------+-------------+----------+--------------------+---------------+---------------+----+-----+-------+----------------+--------------------+----------+
|  14.23|      1.71|2.43|        15.6|      127|          2.8|      3.06|                0.28|           2.29|           5.64|1.04| 3.92|   1065|               1|[14.23,1.71,2.43,...|         2|
|   13.2|      1.78|2.14|        11.2|      100|         2.65|      2.76|                0.26|           1.28|           4.38|1.05|  3.4|   1050|               1|[13.2,1.78,2.14,1...|         2|
|  13.16|      2.36|2.67|

Найдите силуэт с евклидовым расстоянием в квадрате для данных по вину(округлите до четвертого знака).
<br><a href="https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.ClusteringEvaluator">ClusteringEvaluator</a>

In [66]:
evaluator = ClusteringEvaluator(predictionCol = "prediction")
evaluator.evaluate(kmmodel.transform(wine_df))

0.7209450863531623

## №3 DecisionTreeClassifier

Загрузити датасет из файла <a href="https://github.com/AlexKbit/stepik-ds-course/raw/master/Week5/SparkML/spark-tasks/iris.parquet">iris.parquet</a>

In [67]:
iris_df = spark.read.parquet("./iris.parquet")

In [68]:
iris_df.show(5)

+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|species|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| setosa|
|         4.9|        3.0|         1.4|        0.2| setosa|
|         4.7|        3.2|         1.3|        0.2| setosa|
|         4.6|        3.1|         1.5|        0.2| setosa|
|         5.0|        3.6|         1.4|        0.2| setosa|
+------------+-----------+------------+-----------+-------+
only showing top 5 rows



Составьте из наших признаков вектор применив <a href="https://spark.apache.org/docs/latest/ml-features.html#vectorassembler">VectorAssembler</a> как колонку features.
<br> Задействуйте все признаки, кроме species, так как он является целевым.

In [71]:
features = VectorAssembler(
inputCols=iris_df.columns[:-1],
outputCol="features")
iris_df = features.transform(iris_df)

Используйте <a href="https://spark.apache.org/docs/latest/ml-features.html#stringindexer">StringIndexer</a> и сделайте новый признак с именем type из целевого признака species который является категориальным.

In [75]:
iris_df = StringIndexer(inputCol="species", outputCol="type").fit(iris_df).transform(iris_df)

In [76]:
iris_df.show(5)

+------------+-----------+------------+-----------+-------+-----------------+----+
|sepal_length|sepal_width|petal_length|petal_width|species|         features|type|
+------------+-----------+------------+-----------+-------+-----------------+----+
|         5.1|        3.5|         1.4|        0.2| setosa|[5.1,3.5,1.4,0.2]| 0.0|
|         4.9|        3.0|         1.4|        0.2| setosa|[4.9,3.0,1.4,0.2]| 0.0|
|         4.7|        3.2|         1.3|        0.2| setosa|[4.7,3.2,1.3,0.2]| 0.0|
|         4.6|        3.1|         1.5|        0.2| setosa|[4.6,3.1,1.5,0.2]| 0.0|
|         5.0|        3.6|         1.4|        0.2| setosa|[5.0,3.6,1.4,0.2]| 0.0|
+------------+-----------+------------+-----------+-------+-----------------+----+
only showing top 5 rows



Сформируем выборки на обучение и тест.

In [77]:
(training_data, test_data) = iris_df.randomSplit([0.8, 0.2],seed = 42)

Создайте и обучите <a href="https://spark.apache.org/docs/latest/ml-classification-regression.html#decision-tree-classifier">DecisionTreeClassifier</a> на датасете для обучения.
Полученную модель используйте над тестовым датасетом.

In [79]:
from pyspark.ml.classification import DecisionTreeClassifier
dtc = DecisionTreeClassifier(featuresCol="features", labelCol="type")
dtcModel = dtc.fit(training_data)
pred = dtcModel.transform(test_data)

Используйте <a href="https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.MulticlassClassificationEvaluator">MulticlassClassificationEvaluator</a> (помните что целевая фича это - type) для оценки качества модели по метрике accuracy.<br>Какая точность полученной модели?

In [81]:
MulticlassClassificationEvaluator(labelCol="type", metricName="accuracy").evaluate(pred)

1.0

## №4 Random forest

Создайте и обучите <a href="https://spark.apache.org/docs/latest/ml-classification-regression.html#random-forest-classifier">RandomForestClassifier</a> из 10 деревьев на датасете для обучения.
Полученную модель примените к тестовому датасету.


In [82]:
from pyspark.ml.classification import RandomForestClassifier
dtc = RandomForestClassifier(featuresCol="features", labelCol="type")
dtcModel = dtc.fit(training_data)
pred = dtcModel.transform(test_data)

Используйте <a href="https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.MulticlassClassificationEvaluator">MulticlassClassificationEvaluator</a> (помните что целевая фича это - type) для оценки качества модели по метрике accuracy.<br>Какая точность полученной модели?

In [83]:
MulticlassClassificationEvaluator(labelCol="type", metricName="accuracy").evaluate(pred)

0.9583333333333334

## №5 Hyperparameter tuning

Займемся оптимизацией гиперпараметров для модели.
Примените <a href='https://spark.apache.org/docs/latest/ml-tuning.html#train-validation-split'>TrainValidationSplit</a> для оптимизации гиперпараметров используя подготовленный вами выше датасет <a href="https://github.com/AlexKbit/stepik-ds-course/raw/master/Week5/SparkML/spark-tasks/iris.parquet">iris.parquet</a> на модели <a href="https://spark.apache.org/docs/latest/ml-classification-regression.html#random-forest-classifier">RandomForestClassifier</a> совместно с <a href="https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.MulticlassClassificationEvaluator">MulticlassClassificationEvaluator</a>.
<br>Ваша цель определить оптимальные значения параметров из следующих диапазонов:
<br>impurity = ["entropy", "gini"]
<br>maxDepth = [2, 3, 4, 5]
<br>numTrees = [3, 6, 9, 12, 15, 18, 21]

In [84]:
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

In [93]:
iris_df = iris_df.withColumnRenamed("type", "label")
rf = RandomForestClassifier(featuresCol="features", labelCol="label")

model = TrainValidationSplit(
    estimator=rf,
    estimatorParamMaps=ParamGridBuilder()\
        .addGrid(rf.impurity, ["entropy", "gini"])\
        .addGrid(rf.maxDepth, [2,3,4,5])\
        .addGrid(rf.numTrees, [3,6,9,12,15,18,21])
        .build(),
    evaluator = MulticlassClassificationEvaluator()
).fit(iris_df)

In [94]:
print('Num Trees: {}'.format(model.bestModel._java_obj.getNumTrees()))
print('Max Depth: {}'.format(model.bestModel._java_obj.getMaxDepth()))
print('Impurity: {}'.format(model.bestModel._java_obj.getImpurity()))

Num Trees: 9
Max Depth: 3
Impurity: entropy
